In [3]:
from google.colab import files
import zipfile
import os


print("⬆️ Upload train.zip")
files.upload()
with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall("Task_B/train")



⬆️ Upload train.zip


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Upload val.zip (ONLY the contents of val/)
print("⬆️ Upload val.zip")
files.upload()
with zipfile.ZipFile("val.zip", 'r') as zip_ref:
    zip_ref.extractall("Task_B/val")

# Verify both folders
print("✅ Extracted folders:")
print("Train:", os.listdir("Task_B/train")[:3])
print("Val:", os.listdir("Task_B/val")[:3])

In [ ]:
import shutil
import os

# Flatten the train/train → train
if os.path.exists("Task_B/train/train"):
    shutil.move("Task_B/train/train", "Task_B/train_temp")
    shutil.rmtree("Task_B/train")
    shutil.move("Task_B/train_temp", "Task_B/train")

# Flatten the val/val → val
if os.path.exists("Task_B/val/val"):
    shutil.move("Task_B/val/val", "Task_B/val_temp")
    shutil.rmtree("Task_B/val")
    shutil.move("Task_B/val_temp", "Task_B/val")

# Confirm
print("✅ Folder structure fixed!")
print("Train samples:", os.listdir("Task_B/train")[:3])
print("Val samples:", os.listdir("Task_B/val")[:3])


✅ Folder structure fixed!
Train samples: ['Charlie_Hunnam', 'Tim_Curry', '048_frontal']
Val samples: ['Lili_Marinho', 'Hans_Eichel', '112_frontal']


In [ ]:
!pip install -q face_recognition
!pip install -q scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install deepface
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=ac5d67670749a5cb3cabe5bcfbe507d878143ec732bb6d16afbf5fd794385b7c
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import os
import numpy as np
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


25-06-29 14:42:44 - Directory /root/.deepface has been created
25-06-29 14:42:44 - Directory /root/.deepface/weights has been created


In [ ]:
def collect_dataset(folder):
    data = []
    for person_folder in os.listdir(folder):
        person_path = os.path.join(folder, person_folder)
        if not os.path.isdir(person_path):
            continue

        for f in os.listdir(person_path):
            if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                data.append((os.path.join(person_path, f), person_folder))

        distortion_folder = os.path.join(person_path, "distortion")
        if os.path.exists(distortion_folder):
            for f in os.listdir(distortion_folder):
                if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                    data.append((os.path.join(distortion_folder, f), person_folder))
    return data


In [ ]:
def extract_embeddings(data_list, model_name="ArcFace"):
    embeddings = []
    labels = []
    for img_path, label in data_list:
        try:
            emb = DeepFace.represent(img_path=img_path, model_name=model_name, enforce_detection=False)[0]['embedding']
            embeddings.append(emb)
            labels.append(label)
        except Exception as e:
            print(f"⚠️ Skipping {img_path} due to error: {e}")
    return np.array(embeddings), labels


In [ ]:
def predict(val_emb, train_embeddings, train_labels):
    sims = cosine_similarity([val_emb], train_embeddings)
    index = np.argmax(sims)
    return train_labels[index]

In [ ]:
def evaluate(train_folder, val_folder):
    print("Collecting training data...")
    train_data = collect_dataset(train_folder)

    print("Collecting validation data...")
    val_data = collect_dataset(val_folder)

    print("Extracting embeddings for training data...")
    train_embeddings, train_labels = extract_embeddings(train_data)

    print("Predicting validation set...")
    y_true = []
    y_pred = []

    for img_path, true_label in val_data:
        try:
            val_emb = DeepFace.represent(img_path=img_path, model_name="ArcFace", enforce_detection=False)[0]['embedding']
            predicted_label = predict(val_emb, train_embeddings, train_labels)
            y_true.append(true_label)
            y_pred.append(predicted_label)
        except Exception as e:
            print(f"Skipping {img_path} due to error during prediction: {e}")

    if len(y_true) == 0 or len(y_pred) == 0:
        print("❗ No predictions made. Dataset may be empty or corrupt.")
    else:
        print("\n--- Evaluation Results ---")
        print("Top-1 Accuracy:", accuracy_score(y_true, y_pred))
        print("Macro F1-Score:", f1_score(y_true, y_pred, average='macro'))
        print("Precision:", precision_score(y_true, y_pred, average='macro'))
        print("Recall:", recall_score(y_true, y_pred, average='macro'))

train_folder = "Task_B/train"
val_folder = "Task_B/val"

evaluate(train_folder, val_folder)

Extracting embeddings for training data...
Predicting validation set...
❗ No predictions made. Dataset may be empty or corrupt.


In [ ]:
# ✅ STEP 4: Define face recognition and evaluation logic
from deepface import DeepFace
import os
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

def get_person_images(data_dir):
    images = []
    labels = []
    for person in os.listdir(data_dir):
        person_folder = os.path.join(data_dir, person)
        if not os.path.isdir(person_folder):
            continue

        for file in os.listdir(person_folder):
            file_path = os.path.join(person_folder, file)
            if file.endswith(('.jpg', '.jpeg', '.png')):
                images.append(file_path)
                labels.append(person)

            # Include distorted images if available
            distortion_folder = os.path.join(person_folder, "distortion")
            if os.path.exists(distortion_folder):
                for dimg in os.listdir(distortion_folder):
                    if dimg.endswith(('.jpg', '.jpeg', '.png')):
                        images.append(os.path.join(distortion_folder, dimg))
                        labels.append(person)

    return images, labels

In [ ]:
# ✅ STEP 5: Load training embeddings
print("\U0001F50D Loading training faces and extracting embeddings using ArcFace...")
try:
    embeddings = DeepFace.find(
        img_path=None,
        db_path=train_folder,
        model_name="ArcFace",
        enforce_detection=False,
        detector_backend="retinaface"
    )
except Exception as e:
    print("\u274C Error while embedding training data:", e)


🔍 Loading training faces and extracting embeddings using ArcFace...
25-06-29 15:03:07 - Found 15408 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:   0%|          | 0/15408 [00:00<?, ?it/s]

25-06-29 15:03:11 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
  0%|          | 524k/119M [00:00<00:47, 2.51MB/s]
  2%|▏         | 2.62M/119M [00:00<00:11, 9.76MB/s]
  8%|▊         | 9.44M/119M [00:00<00:03, 30.5MB/s]
 15%|█▌        | 17.8M/119M [00:00<00:02, 47.7MB/s]
 22%|██▏       | 26.2M/119M [00:00<00:01, 57.2MB/s]
 29%|██▉       | 34.6M/119M [00:00<00:01, 64.2MB/s]
 36%|███▌      | 43.0M/119M [00:00<00:01, 68.7MB/s]
 43%|████▎     | 51.4M/119M [00:00<00:00, 71.8MB/s]
 50%|█████     | 59.8M/119M [00:01<00:00, 74.1MB/s]
 57%|█████▋    | 68.2M/119M [00:01<00:00, 75.7MB/s]
 65%|██████▍   | 76.5M/119M [00:01<00:00, 76.6MB/s]
 72%|███████▏  | 84.9M/119M [00:01<00:00, 77.1MB/s]
 79%|███████▊  | 93.3M/119M [00:01<00:00, 77.6MB/s]
 85%|████████▌ | 101M/119M [00:01<00:00, 76.9MB/s] 
 92%|█████████▏| 110M/119M [00:01<00:00, 76.1MB/s]
100%|██████████| 119M/119M [00:01<

KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm

def predict_and_evaluate(val_folder, db_path):
    y_true = []
    y_pred = []

    for person in os.listdir(val_folder):
        person_folder = os.path.join(val_folder, person)
        if not os.path.isdir(person_folder):
            continue

        for file in os.listdir(person_folder):
            file_path = os.path.join(person_folder, file)
            if file_path.endswith(('.jpg', '.jpeg', '.png')):
                try:
                    result = DeepFace.find(
                        img_path=file_path,
                        db_path=db_path,
                        model_name="ArcFace",
                        enforce_detection=False,
                        detector_backend="retinaface",
                        silent=True
                    )

                    # Predicted folder name is in identity path
                    if len(result[0]) > 0:
                        pred_identity = result[0].iloc[0]["identity"]
                        predicted_label = pred_identity.split(os.path.sep)[-2]  # Extract folder name
                    else:
                        predicted_label = "Unknown"
                except:
                    predicted_label = "Unknown"

                y_true.append(person)
                y_pred.append(predicted_label)

In [ ]:
# 4. Install DeepFace and required packages
!pip install deepface --quiet
import os
from deepface import DeepFace
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import glob
import cv2
from tqdm import tqdm


In [ ]:
# 5. Helper function to load images and labels
def load_dataset(folder_path):
    image_paths = []
    labels = []
    subfolders = sorted(os.listdir(folder_path))
    for person in subfolders:
        person_folder = os.path.join(folder_path, person)
        if os.path.isdir(person_folder):
            # Main image (e.g., John.jpg)
            for img in glob.glob(os.path.join(person_folder, "*.jpg")):
                image_paths.append(img)
                labels.append(person)
            # Distorted images
            distortion_path = os.path.join(person_folder, "distortion")
            if os.path.exists(distortion_path):
                for img in glob.glob(os.path.join(distortion_path, "*.jpg")):
                    image_paths.append(img)
                    labels.append(person)
    return image_paths, labels


In [ ]:
# 6. Embed faces using DeepFace
def extract_embeddings(image_paths):
    embeddings = []
    for img_path in tqdm(image_paths, desc="🔍 Extracting embeddings"):
        try:
            embedding = DeepFace.represent(
                img_path=img_path,
                model_name='ArcFace',
                detector_backend='opencv',
                enforce_detection=False
            )[0]['embedding']
            embeddings.append(embedding)
        except Exception as e:
            print(f"⚠️ Skipping {img_path}: {e}")
            embeddings.append(None)
    return embeddings


In [ ]:
# 7. Evaluation logic
from sklearn.neighbors import KNeighborsClassifier

def evaluate(train_folder, val_folder):
    print("📂 Loading training data...")
    train_images, train_labels = load_dataset(train_folder)
    print("📂 Loading validation data...")
    val_images, val_labels = load_dataset(val_folder)

    print(f"🔢 Training samples: {len(train_images)}, Validation samples: {len(val_images)}")

    train_embeddings = extract_embeddings(train_images)
    val_embeddings = extract_embeddings(val_images)

    # Filter None values
    train_data = [(emb, label) for emb, label in zip(train_embeddings, train_labels) if emb is not None]
    val_data = [(emb, label) for emb, label in zip(val_embeddings, val_labels) if emb is not None]

    if len(train_data) == 0 or len(val_data) == 0:
        print("❌ No valid data for training/validation")
        return

    X_train, y_train = zip(*train_data)
    X_val, y_val = zip(*val_data)

    clf = KNeighborsClassifier(n_neighbors=1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)

    print("\n--- 📊 Evaluation Results ---")
    print("Top-1 Accuracy:", accuracy_score(y_val, y_pred))
    print("Macro F1-Score:", f1_score(y_val, y_pred, average='macro'))
    print("Precision:", precision_score(y_val, y_pred, average='macro'))
    print("Recall:", recall_score(y_val, y_pred, average='macro'))


In [ ]:
# 8. Call everything
train_path = "Task_B/train"
val_path = "Task_B/val"
evaluate(train_path, val_path)


📂 Loading training data...
📂 Loading validation data...
🔢 Training samples: 15408, Validation samples: 3376


🔍 Extracting embeddings: 100%|██████████| 3376/3376 [51:27<00:00,  1.09it/s]



--- 📊 Evaluation Results ---
Top-1 Accuracy: 0.0
Macro F1-Score: 0.0
Precision: 0.0
Recall: 0.0
